## 自监督卷积子空间聚类网络  
  
### 摘要  
  
  基于数据self-expression性质的子空间聚类方法变的十分流行。它从低维线性子空间的并集中学习。  
    
 然而子空间聚类的适用性很受限，因为实际上原生数据不一定位于这样的线性子空间中。另一方面尽管卷积神经网络体现了其提取可识别特征的强大能力，但是训练这样的一个网络需要大量的有标签的数据，而这些标签在子空间聚类应用中通常是难以获得的。  
   
**为了实现的同步的特征学习和子空间聚类，本文提出了一个端到端的训练框架（叫做自监督卷积子空间聚类神经网络），这个网络结合了卷积神经网络（用于特征学习），和 self-expression模型（用于子空间聚类），和谱聚类模型（用于自监督），把这3个模型联合进一个优化框架中**  
  
特别的是，我们引入了一个二元的自监督，它能利用谱聚类的输出去监督特征学习的训练（通过一个分类损失），监督self-expression（通过谱聚类的损失）  
  
<br>  

### 引言  

在很多应用中我们需要处理大量的高维数据，**这些高维数据通常被视作多个低维子空间的并集。每个子空间通常对应一个分类。**  
  
例如：一个对象在各种光照条件下的正面照近似为跨度为9个维度的线性子空间。视频序列中与刚性运动对象相关的特征点的轨迹跨越一个高达3维的仿射子空间。在手写数字集中，单个数字也只是跨度在一个低维子空间中。  
  
在这些例子中，最重要的是把数据分割为多个组，每个组都只包含了来自同一个子空间的数据点。   
  
<br>  

<br>  

**子空间聚类的问题定义如下：  
让$X\in R^{D\times N}$,D是全维度数，N是数据样本个数，X的每一列都是来自于n个子空间的并集，每个子空间是$R^D$的子集，$U_i^n\{s_i\}$。每个子空间的维度$d_i<<min\{D,N\} \quad$ .(其实质就是对每列样本只取这列样本所属类别线性相关属性所在的维度。其他维度为0，N个样本都这样做，最后组成了X数据矩阵) 。 子空间聚类的目标就是把X的列分割为相应的子空间。**  
  
<br>  

在过去几十年，子空间聚类是无监督学习的一个重要课题，且研发了很多子空间聚类算法。这些方法成功的应用于动作分割，人脸聚类，基因表示阵列聚类等等。  
  
尽管近年来子空间聚类的发展取得了很大的成功，但由于实际数据不一定符合线性子空间模型，其在实际应用中的适用性非常有限。例如，在人脸图像聚类中，实际的人脸图像通常是不对齐的，并且常常包含对象的姿势和表情的变化。    
  
**子空间聚类不能处理同一类图片不位于同一子空间的情况。（子空间聚类无法处理同一人脸对应的图像不再位于线性子空间的情况。）**  
  
**虽然最近发展了联合图像对齐和子空间聚类的技术[21]，但这种参数化模型无法处理更广泛的数据变化，如变形、平移等。**  
   
**在执行子空间聚类之前，例如在[37，36]中，也可以使用诸如SIFT[25]、HOG[1]和PRICoLBP[39]等论文里提出的人工设计的不变的特征。**  
  
然而，目前还没有理论和实践证据表明这些特征遵循线性子空间模型。  
  
最近卷积神经网络证明了其在学习有用的图片特征上的优越性能。  
  
**卷积神经网络能够从不同子空间的并集中学习特征**  
  
训练卷积神经网络的困难在于需要大量的有标签的数据集，这在实际数据集上很难用到。    

为了使用无标签数据来训练卷积神经网路去学习特征。最近很多方法提出使用子空间并集数据的self-expression性质。  
  
这些方法监督了卷积神经网络的训练，通过学习特征向量，每个特征向量可能表示为其他特征向量的线性组合。  
  
 这种方法由于缺少有效的监督信息很难学到好的特征表示。  
   
 **本文的贡献：**  
 本文提出了一个端到端的同时进行特征学习和子空间聚类的训练框架，称为自监督卷积子空间聚类网络（S2ConvSCN）。  
 **利用当前的聚类结果对特征学习和自表达模块的训练进行自我监督，能够显著提高子空间聚类的性能。**  
 我们特别介绍了**以下两个自我监控模块：  
 1、我们引入了一个谱聚类模块，它利用当前的聚类结果来监督自表达系数的学习。这是通过诱导selfexpression生成的亲和力来形成数据的分段来实现的，该分段与集群生成的当前类标签一致。  
 2、我们引入了一个分类模块，利用当前的聚类结果来监督特征学习的训练。这是通过最小化在特征学习模块上训练的分类器的输出和由聚类生成的当前类标签之间的分类损失来实现的。**  
   
 我们提出了一个训练框架，在学习过程中，特征表示、数据自表达和数据分割被联合学习和交替地优化。从概念上讲，初始的聚类结果与真实的数据分割并不完全一致，因此初始的自我监督会给训练带来错误。尽管如此，**特征学习仍然有望从这种自我监督中受益，因为有正确标签的数据可以产生有用的信息。一个有效的特征随后会帮助学习更高的self-expression和数据分割**  
   
<br>  
  
### 回顾相关工作    

我们回顾了子空间聚类的相关前期工作。为了清晰起见，我们将其分为两类：a）原始空间中的子空间聚类；b）特征空间中的子空间聚类。
    
#### 1、原始空间中的子空间聚类    
  在过去的几年中，子空间聚类得到了广泛的关注，并发展了许多方法。这些算法中基于谱聚类的算法最流行。  
  **这些算法把子空间聚类的问题划分为两个子问题：子问题1、从原始数据空间学习亲和力矩阵。子问题2：在亲和力矩阵上应用谱聚类算法去找到最优的数据的划分。**  
    
  找到有效的亲和力矩阵是最关键的一步。  
    
 亲和力矩阵的计算是基于数据的self-expression性质的。这个性质是指子空间并集中的数据点可以表示为其他数据点的线性组合，$x_j=\sum_{i\neq j} C_{ij}X_i +e_j$  这里ej是噪声。预计数据点xj的线性组合将与xj属于同一子空间的数据点组合在一起。为了达到这一目的，在线性组合系数上使用了不同类型的正则化项。例如1范数正则化项，如系数矩阵的核正则化项用来找到特征的低秩表示，如用1范数与2范数或核范数的混合来平衡线性组合系数的稀疏性和稠密性。如使用了一种依赖于数据的稀疏诱导正则化器来寻找稀疏线性组合。另一方面，也研究了不同的方法来模拟数据中的噪声或损坏，[2]中使用了向量1范数。采用2,1范数，[9]采用了相关熵项
  
<br>  

#### 2、特征空间中的子空间聚类  
对于特征空间中的子空间聚类，我们进一步将现有的方法分为两类。  
1、第一种类型使用潜在特征空间，它是通过Mercer核诱导的，或者通过矩阵分解来构造。  
2、第二种类型使用显式特征空间，它是通过人工特征提取设计的，例如[37]，或者是从数据中学习的  
* 潜在子空间  
    近年来，许多研究利用核技巧将原始数据映射到一个高维的潜在特征空间，在该空间中进行子空间聚类。例如，在核稀疏子空间聚类方法[35,33]和核低秩表示方法[31,48,12]中使用了预定义多项式和高斯核。遗憾的是，这种预定义核函数所诱导的潜在特征空间中的数据不一定存在于低维子空间中
  。另一方面，通过矩阵分解构造了潜在特征空间，在[24]中，线性变换矩阵和低秩表示同时计算；在[34]中，线性变换和稀疏表示联合优化。然而，学习的线性变换的表示能力仍然有限。  
      
      
* 显式特征空间  
   深度学习以一种端到端可训练的方式学习层次特征的强大能力，引起了广泛的研究兴趣。近年来，在子空间聚类中，利用深度学习技术进行特征提取的研究较少。或者举例来说，在[37，36]中，开发了一个完全连接的深度自动编码器网络，该网络具有手工制作的特征（例如SIFT或HOG特征）与稀疏的自表达模型相结合；在文献[14]中，提出了一种具有加号自表达模型的叠层卷积自动编码器网络。  
   **尽管已有研究表明，这些方法在聚类精度上有很大的潜力，但由于没有将聚类结果中潜在有用的监督信息纳入特征学习步骤，也没有开发出一个将特征学习与子空间聚类充分结合的联合优化框架。**  
   最近，在[54]中，在[14]的框架中采用了具有子空间特定生成器和子空间特定鉴别器的深度对抗网络来进行子空间聚类。  
   然而，鉴别器需要使用每个子空间的维数，这通常是未知的。  
   本文试图建立一个结合特征学习和子空间聚类的联合优化框架，以便从子空间聚类结果中获得有用的自我监督信息来指导特征学习和细化自表达模型。  
   受近年来卷积神经网络在图像和视频数据集分类任务中的成功[15]和最近的工作[14]的启发，我们将卷积特征提取模块集成到子空间聚类中，形成了一个端到端可训练的联合优化框架，称为自监督卷积子空间聚类网络  
   在S2ConvSCN中，基于叠层卷积的特征提取和基于自表达的亲和力学习都可以通过反馈光谱聚类得到有效的监控。
<br>  

### 我们的方案：自监督卷积子空间聚类网络   
在这一部分，我们将介绍我们的S2ConvSCN用于联合特征学习和子空间聚类。我们首先介绍我们的网络公式（见图1），然后介绍自我监督模块。最后，我们提出了一个有效的训练方法。
  
  如前所述，我们的网络由一个特征提取模块、一个自表达模块和一个用于训练前两个模块的自监督模块组成。   
    
*  特征提取模块。  
    我们提出的S2ConvSCN的一个基本组成部分是特征提取模块，用于从原始数据中提取适合于子空间聚类的特征。为了在保持空间局部性的同时提取局部特征，我们采用了由多个卷积层组成的卷积神经网络。我们将网络的输入表示为h（0）=x，其中x是图像。卷积层ξ包含一组滤波器w（ξ）i和相关的偏差b（ξ）i，i=1，···，m（ξ），并根据前一层的输出生成m（ξ）特征图。然后使用网络顶层L中的特征映射{h（L）i}i=1，···，m（L）来形成输入数据x的表示。具体地，m（L）特征映射{h（L）i}m（L）i=1被矢量化并连接以形成表示向量z，即。，  
    $Z=[h_1^L(:),……，h_m^L(:)]$  
    其中h（L）1（：），···，h（L）m（L）（：）是表示特征地图h（L）1，···，h（L）m（L）的矢量化的行向量。这些向量被水平连接，然后被转置以形成向量z。  
     **为了确保学习的表示z包含来自输入数据x的有意义的信息，特征映射h（L）1，···，h（L）m（L）被输入到解码器网络中以重建图像ˆx。**   
     此编码器编码器网络的损失函数为重建错误：  
     ![](img/1.PNG)  
     其中N是训练集中的图像数.  
       
<br>  
*  自监督模块
    一旦得到自我表现系数矩阵C，我们就可以将数据亲和矩阵计算为$A=\frac{1}{2}(|C|+|C^T|)$随后，可以对A应用谱聚类以通过最小化以下成本来获得数据的分段：  
    ![](img/2.PNG)  
    观察到谱聚类产生了一个数据集的标签，尽管它不一定是所有数据点的正确的类标签，但它包含了关于数据的有意义的信息。这促使我们利用光谱聚类的输出来监督特征提取和自表达模块的训练。原则上，从特征提取模块学习到的特征应该包含足够的信息来预测数据点的类标签。因此，我们在特征提取模块的顶部引入了一个分类层，该模块期望生成与光谱聚类中生成的标签一致的标签。此外，该分割产生了光谱聚类分析法，用于构造二元分割矩阵，该矩阵包含关于在表示特定数据点时应使用哪些数据点的信息。因此，我们将谱聚类的目标函数作为一个损失函数加入到我们的网络规划中，从而起到监督自表达模块训练的作用。我们将在以下两个小节中详细介绍这两个自我监督模块。  
      
<br>  

#### 自我表达的自监督  
  为了充分利用光谱聚类产生的标签信息，我们将谱聚类作为网络的一个模块，为自表达模型提供反馈。为了了解（4）中光谱聚类的目标函数是如何提供这种反馈的，我们将（4）改写为C的加权1范数，如[17]所示，即：，![](img/3.PNG)
其中我们使用了aij=12（| cij |+| cji |）。由（5）可以看出，KCKQ测量了系数smatrixc与分割矩阵Q之间的差异。  
当提供Q时，最小化成本kckq具有强制自表达式矩阵C的效果，使得仅当第i和第j个数据点具有相同的类标签时，条目ciji非零。因此，将kCkQin这一术语纳入网络公式有助于自我表达模块的训练。也就是说，可以将先前的谱聚类结果融入到自表达模型中，为细化自表达矩阵C提供自监督。
<br>  
  
#### 特征学习自我监督    
  我们还利用光谱聚类产生的类标签来监督特征提取模块的训练。注意，谱聚类的输出是一个n维向量，它表示n个子空间（即簇）的隶属度。因此，我们将FC层设计为p×N1×N2×n，其中p是提取的卷积特征的维数，定义为编码器块中最后一个卷积层的不同特征映射的连接，N1和N2分别是两个FC层中的神经元数目。因此，我们将FC层设计为p×N1×N2×n，其中p是提取的卷积特征的维数，定义为编码器块中最后一个卷积层的不同特征映射的连接，N1和N2分别是两个FC层中的神经元数目。为了利用自监督信息来训练卷积编码器，我们定义了交叉熵损失和中心损失（CEC）的混合，如下所示：  
  ![](img/4.PNG)  
  式中，yjis是yjvia softmax的标准化，μπ（yj）表示与yj相对应的簇中心，π（yj）取谱聚类输出的yjj指标，0≤τ≤1为折衷参数。L4的第一项是交叉熵损失，第二项是压缩簇内变化的中心损失。
  定义这种损失函数的一个重要问题是谱聚类{qj}nj=1的输出仅为输入数据提供伪标签。也就是说，在谱聚类的返回结果中，分配给一个簇的标签索引是一个未知的置换。因此，两个连续时代的类标签可能不一致。为了解决这个问题，我们建议通过匈牙利算法[29]对新的伪标签进行置换，以在连续迭代的伪标签之间找到一个最佳分配，然后将它们输入到具有交叉熵损失的selfsupervision模块中备注1。注意，谱聚类的输出用于两个相互关联的自监督模块，因此我们称之为双重自监督机制
<br>    

![](img/5.PNG)
  
<br>  

<br>  

<br>     

<br>  

<br> 

### 火花   
  
  **为了实现的同步的特征学习和子空间聚类，本文提出了一个端到端的训练框架（叫做自监督卷积子空间聚类神经网络），这个网络结合了卷积神经网络（用于特征学习），和 self-expression模型（用于子空间聚类），和谱聚类模型（用于自监督），把这3个模型联合进一个优化框架中**    
    
  **子空间聚类不能处理同一类图片不位于同一子空间的情况。（子空间聚类无法处理同一人脸对应的图像不再位于线性子空间的情况。）**   
    
  **虽然最近发展了联合图像对齐和子空间聚类的技术[21]，但这种参数化模型无法处理更广泛的数据变化，如变形、平移等。**    
    
   **在执行子空间聚类之前，例如在[37，36]中，也可以使用诸如SIFT[25]、HOG[1]和PRICoLBP[39]等论文里提出的人工设计的不变的特征。**   
     
  **最近卷积神经网络证明了其在学习有用的图片特征上的优越性能。卷积神经网络能够从不同子空间的并集中学习特征。但是需要大量有监督的样本**   
    
  <br>  
  
  **子空间聚类：需要有效的子空间特征。（因为实际数据可能不符合线性子空间模型）。卷积神经网络：从不同子空间的并集中学习有效特征。但是需要大量有监督样本。 现在的思路：子空间聚类，使用卷积神经提供的特征。 但是要解决需要有监督样本。**  
    
  **self-expression性质无监督学习特征：x=xc ,学习特征向量，每个特征向量可能表示为其他特征向量的线性组合。每个样本学到的特征向量为XC。这种方法由于缺少有效的监督信息很难学到好的特征表示。**  
    
  